In [1]:
import pandas as pd
import numpy as np

In [2]:
details=pd.read_csv('movie_details.csv')
details.drop('Unnamed: 0',axis=1,inplace=True)
ratings=pd.read_csv('movie_ratings.csv')
ratings.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
users=pd.read_csv('movie_users.csv')
users.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
data1=pd.merge(details,ratings)

In [5]:
data2=pd.merge(data1,users)

In [6]:
data2.head()

,영화아이디,제목,장르,사용자아이디,평점,타임스탬프,성별,연령,직업,지역
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


# 여자들이 좋아하는 영화의 장르 찾기
- 기준: 여성 평균 평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화
- 최종 결과: 평점 기준 상위 10위 장르

In [7]:
female=data2[data2['성별']=='F']

In [8]:
# 평점의 평균이 4.0 이상, 개수가 500개 이상
# 기준값 : 영화당 평점의 평균,개수
tmp=female.pivot_table(index='제목',values='평점',aggfunc=['mean','count']).swaplevel(0,1,axis=1)

In [9]:
tmp.head()

평점      
                                   mean count
제목                                           
$1,000,000 Duck (1971)         3.375000    16
'Night Mother (1986)           3.388889    36
'Til There Was You (1997)      2.675676    37
'burbs, The (1989)             2.793478    92
...And Justice for All (1979)  3.828571    35

In [12]:
# 제목으로 index를 지정한 경우
# 제목 값을 기준으로 장르 값을 찾아야한다.
female_movies=tmp[(tmp[('평점','mean')]>=4.0) & (tmp[('평점','count')]>=500)].sort_values(('평점','mean'),ascending=False).index
female_movie_df=tmp[(tmp[('평점','mean')]>=4.0) & (tmp[('평점','count')]>=500)].sort_values(('평점','mean'),ascending=False)

In [13]:
# 공통 컬럼으로 결합 -> merge
tmp1=female_movie_df.reset_index()

In [14]:
tmp1.shape

(22, 3)

In [69]:
tmp1.merge(details,on='제목')[['제목','장르']].head(10)

,제목,장르
0,Schindler's List (1993),Drama|War
1,"Shawshank Redemption, The (1994)",Drama
2,"Sixth Sense, The (1999)",Thriller
3,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
4,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
5,Raiders of the Lost Ark (1981),Action|Adventure
6,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
7,Casablanca (1942),Drama|Romance|War
8,"Silence of the Lambs, The (1991)",Drama|Thriller
9,American Beauty (1999),Comedy|Drama


In [71]:
# 처음부터 장르 값 가져오기
tmp2=female.pivot_table(index='장르',values='평점',aggfunc=['mean','count'])

In [73]:
tmp2.head()

,mean,count
,평점,평점
장르,,
Action,3.367474,1611
Action|Adventure,3.701213,1978
Action|Adventure|Animation,3.843750,64
Action|Adventure|Animation|Children's|Fantasy,3.146341,41
Action|Adventure|Animation|Horror|Sci-Fi,3.422535,71


In [153]:
tmp2 = female.pivot_table(index='장르', values='평점',
                       aggfunc=['mean','count'])

In [154]:
tmp2.head()

,mean,count
,평점,평점
장르,,
Action,3.367474,1611
Action|Adventure,3.701213,1978
Action|Adventure|Animation,3.843750,64
Action|Adventure|Animation|Children's|Fantasy,3.146341,41
Action|Adventure|Animation|Horror|Sci-Fi,3.422535,71


In [155]:
tmp2.shape

(300, 2)

# 장르 데이터 전처리
- 세부 장르 분석을 위해 문자열에 대한 전처리 수행

In [156]:
# 중복을 제거한 장르 고유값
# set()타입을 활용한 고유값 처리
tmp2_index=tmp2.reset_index()

In [157]:
tmp2_index

,장르,mean,count
,,평점,평점
0,Action,3.367474,1611
1,Action|Adventure,3.701213,1978
2,Action|Adventure|Animation,3.843750,64
3,Action|Adventure|Animation|Children's|Fantasy,3.146341,41
4,Action|Adventure|Animation|Horror|Sci-Fi,3.422535,71
5,Action|Adventure|Children's,1.250000,4
6,Action|Adventure|Children's|Comedy,2.447154,123
7,Action|Adventure|Children's|Fantasy,1.857143,7
8,Action|Adventure|Children's|Sci-Fi,2.163636,55


In [129]:
tmp2_index['장르'].apply(lambda x: x.split('|'))

0                                               [Action]
1                                    [Action, Adventure]
2                         [Action, Adventure, Animation]
3      [Action, Adventure, Animation, Children's, Fan...
4         [Action, Adventure, Animation, Horror, Sci-Fi]
5                        [Action, Adventure, Children's]
6                [Action, Adventure, Children's, Comedy]
7               [Action, Adventure, Children's, Fantasy]
8                [Action, Adventure, Children's, Sci-Fi]
9                            [Action, Adventure, Comedy]
10                    [Action, Adventure, Comedy, Crime]
11                   [Action, Adventure, Comedy, Horror]
12           [Action, Adventure, Comedy, Horror, Sci-Fi]
13                  [Action, Adventure, Comedy, Romance]
14                   [Action, Adventure, Comedy, Sci-Fi]
15                      [Action, Adventure, Comedy, War]
16                            [Action, Adventure, Crime]
17                     [Action,

In [130]:
tmp2_split=tmp2_index['장르'].apply(lambda x: x.split('|'))
tmp2_final=[]
for i in tmp2_split:
    for j in i:
        tmp2_final.append(j)
tmp2_final=set(tmp2_final)


In [131]:
# 강사님 방법
result=tmp2.reset_index().copy()

In [147]:
female_fav_genre=result['장르'].str.split('|',expand=True)

In [148]:
np.unique(female_fav_genre.fillna('-'))[1:]

array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'], dtype=object)

In [177]:
female_fav_genre.tail()

,0,1,2,3,4,5
295,Sci-Fi,Thriller,War,None,None,None
296,Sci-Fi,War,None,None,None,None
297,Thriller,None,None,None,None,None
298,War,None,None,None,None,None
299,Western,None,None,None,None,None


In [183]:
# 가장 인기있는 장르 top3
# sr.value_counts()

In [184]:
cat1=female_fav_genre[0].value_counts()
cat2=female_fav_genre[1].value_counts()
cat3=female_fav_genre[2].value_counts()
cat4=female_fav_genre[3].value_counts()
cat5=female_fav_genre[4].value_counts()
cat6=female_fav_genre[5].value_counts()

In [185]:
cat1+cat2+cat3+cat4+cat5+cat6

Action          NaN
Adventure       NaN
Animation       NaN
Children's      NaN
Comedy          NaN
Crime           NaN
Documentary     NaN
Drama           NaN
Fantasy         NaN
Film-Noir       NaN
Horror          NaN
Musical         NaN
Mystery         NaN
Romance         NaN
Sci-Fi          NaN
Thriller        NaN
War            34.0
Western         NaN
dtype: float64

In [191]:
result=cat1.add(cat2,fill_value=0)
result=result.add(cat3,fill_value=0)
result=result.add(cat4,fill_value=0)
result=result.add(cat5,fill_value=0)
result=result.add(cat6,fill_value=0)
result.sort_values(ascending=False).head(3)

Action       96.0
Adventure    86.0
Comedy       86.0
dtype: float64

In [194]:
# 함수
data=np.unique(female_fav_genre.fillna('-'),return_counts=True)

In [195]:
result=pd.Series(data[1][1:],index=data[0][1:])

In [196]:
result

Action         96
Adventure      86
Animation      29
Children's     55
Comedy         86
Crime          42
Documentary     5
Drama          78
Fantasy        33
Film-Noir      18
Horror         30
Musical        31
Mystery        36
Romance        59
Sci-Fi         69
Thriller       77
War            34
Western        17
dtype: int64

# 연령대 별 영화 평점 분석
- 사용자 정의 함수를 사용하여 처리
- 50대가 가장 좋아하는 영화 top10
    - 평균 평점이 높은 순서로 상위 10개의 제목

In [ ]:
# 연령대
# 연속형 변수를 범주형 변수
# 1. 사용자정의함수
# 2. pd.cut()
# 3. np.digitize()

In [237]:
data2['연령'].describe().astype(int)

count    1000209
mean          29
std           11
min            1
25%           25
50%           25
75%           35
max           56
Name: 연령, dtype: int32

In [246]:
# 사용자 정의 함수
def age_range(x):
    if x<20:
        return '10s'
    elif x<30:
        return '20s'
    elif x<40:
        return '30s'
    elif x<50:
        return '40s'
    else:
        return '50s'

In [247]:
data2['연령대']=data2['연령'].apply(lambda x: age_range(x))

In [255]:
data2.head()

,영화아이디,제목,장르,사용자아이디,평점,타임스탬프,성별,연령,직업,지역,연령대
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067,10s
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067,10s
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067,10s
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067,10s
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067,10s


In [256]:
# pd.cut()
data['연령대2']=pd.cut(data2['연령'],[0,20,30,40,50,60],labels=['10s','20s','30s','40s','50s'])

TypeError: 'tuple' object does not support item assignment

In [254]:
# np.digitize()
# 0: 10대
# 4: 50대
np.digitize(data2['연령'],[20,30,40,50])

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [258]:
# 매핑처리 필요
# 딕셔너리
# 0~4
my_dict={}
for i,j in enumerate(['10s','20s','30s','40s','50s']):
    my_dict[i]=j

In [259]:
my_dict

{0: '10s', 1: '20s', 2: '30s', 3: '40s', 4: '50s'}

In [260]:
data2['연령대'].map(my_dict)

KeyError: '연령3'

In [261]:
# 50대 인기 영화 제목
# 인기: 평균 평점이 높은 순서롤 상위 top10 영화 제목
data2.head()

,영화아이디,제목,장르,사용자아이디,평점,타임스탬프,성별,연령,직업,지역,연령대
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067,10s
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067,10s
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067,10s
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067,10s
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067,10s


In [276]:
age50_data=data2[data2['연령대']=='50s'][['연령대','평점','제목']]

In [277]:
age50_data.head()

,연령대,평점,제목
53,50s,4,Toy Story (1995)
54,50s,4,Sense and Sensibility (1995)
55,50s,4,Babe (1995)
56,50s,5,Pocahontas (1995)
57,50s,5,"Umbrellas of Cherbourg, The (Parapluies de Che..."


In [279]:
# 평균평점순으로 상위 10개
age50_data.pivot_table(index='제목',values='평점',aggfunc='mean').sort_values('평점',ascending=False).head(10)

,평점
제목,
It Takes Two (1995),5.0
"Paris, France (1993)",5.0
Tetsuo II: Body Hammer (1992),5.0
Schlafes Bruder (Brother of Sleep) (1995),5.0
My Life and Times With Antonin Artaud (En compagnie d'Antonin Artaud) (1993),5.0
Lured (1947),5.0
Ed's Next Move (1996),5.0
Smashing Time (1967),5.0
"Apple, The (Sib) (1998)",5.0


In [288]:
tmp2=data2.pivot_table(index='제목',values='평점',columns='연령대',aggfunc='mean')

In [290]:
tmp2['50s'].sort_values(ascending=False).head(10).index

Index(['Among Giants (1998)', 'Tetsuo II: Body Hammer (1992)', 'Lured (1947)',
       'It Takes Two (1995)',
       'Best Man, The (Il Testimone dello sposo) (1997)',
       'All Things Fair (1996)', 'Sunset Strip (2000)', 'Love Serenade (1996)',
       'Mating Habits of the Earthbound Human, The (1998)',
       'Return with Honor (1998)'],
      dtype='object', name='제목')

# 남자와 여자의 호불호가 크게 갈리는 영화 10개 출력
- 전체 평점 개수가 500개 이상인 영화만 대상으로 함
- 호불호: 성별 평점의 차이의 크기가 크다=> 새로운 컬럼으로 저장
- 성별 평점 차이의 크기 기준으로 내림차순 정렬하여 상위 10개 영화의 제목 찾기

In [308]:
data2.head()

,영화아이디,제목,장르,사용자아이디,평점,타임스탬프,성별,연령,직업,지역,연령대
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067,10s
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067,10s
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067,10s
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067,10s
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067,10s


In [342]:
tmp=data2.pivot_table(index='제목',values='평점',aggfunc='count')

In [366]:
movies=tmp[tmp['평점']>=500].index

In [367]:
# 제목 컬럼
# 1. sr.isin()
final1=data2[data2['제목'].isin(movies)]

In [368]:
final1.shape

(594712, 11)

In [369]:
# 2. sr.apply(lambda x : x in [])
data2[data2['제목'].apply(lambda x: x in movies)].shape

(594712, 11)

In [370]:
final1.head()

,영화아이디,제목,장르,사용자아이디,평점,타임스탬프,성별,연령,직업,지역,연령대
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067,10s
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067,10s
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067,10s
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067,10s
6,588,Aladdin (1992),Animation|Children's|Comedy|Musical,1,4,978824268,F,1,10,48067,10s


In [371]:
final2=final1.pivot_table(index='제목',values='평점',columns='성별')

In [379]:
final1.pivot_table(index=['제목','성별'],values='평점').unstack()

평점          
성별                                                         F         M
제목                                                                    
10 Things I Hate About You (1999)                   3.646552  3.311966
101 Dalmatians (1961)                               3.791444  3.500000
12 Angry Men (1957)                                 4.184397  4.328421
13th Warrior, The (1999)                            3.112000  3.168000
20,000 Leagues Under the Sea (1954)                 3.670103  3.709205
2001: A Space Odyssey (1968)                        3.825581  4.129738
28 Days (2000)                                      3.209424  2.977707
Abyss, The (1989)                                   3.659236  3.689507
Ace Ventura: Pet Detective (1994)                   3.000000  3.197917
Addams Family, The (1991)                           3.186170  3.163498
Adventures in Babysitting (1987)                    3.455782  3.208122
Adventures of Buckaroo Bonzai Across the 8th Di...  3.308511  3.402321
African Queen, The (1951)                           4.324232  4.223822
Air Force One (1997)                                3.699588  3.555822
Airplane II: The Sequel (1982)                      2.906250  2.995671
Airplane! (1980)                                    3.656566  4.064419
Aladdin (1992)                                      3.857143  3.756494
Alice in Wonderland (1951)                          3.705882  3.692308
Alien (1979)                                        3.888252  4.216119
Alien Nation (1988)                                 3.433333  3.195946
Alien: Resurrection (1997)                          2.708738  2.997041
Aliens (1986)                                       3.802083  4.186684
Alien� (1992)                                       3.008264  3.042289
Almost Famous (2000)                                4.220217  4.228731
Amadeus (1984)                                      4.346734  4.213415
American Beauty (1999)                              4.238901  4.347301
American Graffiti (1973)                            3.940476  4.055556
American History X (1998)                           4.181818  4.240741
American Pie (1999)                                 3.539792  3.754545
American President, The (1995)                      3.923483  3.718654
...                                                      ...       ...
Wag the Dog (1997)                                  3.573222  3.459581
Waking Ned Devine (1998)                            3.963964  3.839917
War of the Worlds, The (1953)                       3.885246  3.855159
Waterworld (1995)                                   2.593496  2.640152
Wayne's World (1992)                                3.464286  3.646429
Wedding Singer, The (1998)                          3.581081  3.445351
Weird Science (1985)                                3.375000  3.435961
West Side Story (1961)                              4.228374  3.953390
Westworld (1973)                                    3.371795  3.428315
What About Bob? (1991)                              3.429293  3.387722
What's Eating Gilbert Grape (1993)                  3.879668  3.820796
When Harry Met Sally... (1989)                      4.257028  3.987850
While You Were Sleeping (1995)                      3.636066  3.385870
White Men Can't Jump (1992)                         3.028777  3.231061
Who Framed Roger Rabbit? (1988)                     3.569378  3.713251
Whole Nine Yards, The (2000)                        3.296552  3.404814
Wild Things (1998)                                  3.392000  3.459082
Wild Wild West (1999)                               2.275449  2.131973
Willow (1988)                                       3.658683  3.453543
Willy Wonka and the Chocolate Factory (1971)        4.063953  3.789474
Witness (1985)                                      4.115854  3.941504
Wizard of Oz, The (1939)                            4.355030  4.203138
Working Girl (1988)                                 3.606742  3.312500
World Is No

In [373]:
final2.shape

(618, 2)

In [380]:
# 절대값 적용해야됨
final2['남녀 호불호']=abs(final2['F']-final2['M'])

In [381]:
final2.sort_values(by='남녀 호불호',ascending=False).head(10)

성별,F,M,남녀 호불호
제목,,,
Dirty Dancing (1987),3.790378,2.959596,0.830782
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
Dumb & Dumber (1994),2.697987,3.336595,0.638608
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
Grease (1978),3.975265,3.367041,0.608224
Caddyshack (1980),3.396135,3.969737,0.573602
Animal House (1978),3.628906,4.167192,0.538286
"Exorcist, The (1973)",3.537634,4.067239,0.529605
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,0.512885
